# Team Ratings

In [1]:
# Third party libraries
import pandas as pd
import numpy as np

# Local libraries
import Tools.json_utils as ju

FILENAME = "Data/data_2024.json"

In [ ]:
# Currently unused (save for Massey, Colley, Elo Ratings)
tournament = Tournament.Tournament(url=TOURNAMENT_URL,
                                   debug=True)

In [2]:
def set_massey_score_entry(dct: dict, team: str, opponent: str, team_score: int, opponent_score: int):
    """Set Massey score entry

    Args:
        dct (dict): Massey score dictionary
        team (str): current team name
        opponent (str): opponent team name
        team_score (int): current team score
        opponent_score (int): opponent team score 

    Returns:
        dct (dict): Massey score dictionary
    """

    dct["Home"].append(team)
    dct["Away"].append(opponent)
    dct["Home_Score"].append(team_score)
    dct["Away_Score"].append(opponent_score)

    return dct
    

def set_massey_rating_data_frame(filename: str):
    """Set Massey score data frame prior to Massey rating calculation

    Args:
        filename (str): Name of JSON team file

    Returns:
        score_df (pd.DataFrame): Massey score data frame
    """
    
    # Initialize score dictionary
    score_dict = {
        "Home": [],
        "Home_Score": [],
        "Away": [],
        "Away_Score": []
    }
    
    # Read Stats
    teams_df = pd.read_json(filename)
    
    all_teams = list(teams_df.keys())
    
    for team in all_teams:
        
        team_df = teams_df[team]
        
        for i in range(len(team_df["Type"])):
        
            # Non Tournament games
            if team_df["Type"][i] != "NCAA" and team_df["Type"][i] != "CIT" and team_df["Tm"][i] is not None and team_df["Opp"][i] is not None:
        
                # Find which team is home/away (None = home, @ = away, N = neutral/assign home to winner?)
                if team_df["Site"][i] is None:
        
                    # Current team is home team
                    # score_dict["Home"].append(team)
                    # score_dict["Away"].append(team_df["Opponent"][i])
                    # score_dict["Home_Score"].append(int(team_df["Tm"][i]))
                    # score_dict["Away_Score"].append(int(team_df["Opp"][i]))
                    score_dict = set_massey_score_entry(dct=score_dict, team=team, opponent=team_df["Opponent"][i],
                                                        team_score=int(team_df["Tm"][i]), opponent_score=int(team_df["Opp"][i]))
                    
                elif team_df["Site"][i] == "@":
        
                    # Opponent team is away team
                    # score_dict["Home"].append(team_df["Opponent"][i])
                    # score_dict["Away"].append(team)
                    # score_dict["Home_Score"].append(int(team_df["Opp"][i]))
                    # score_dict["Away_Score"].append(int(team_df["Tm"][i]))
                    score_dict = set_massey_score_entry(dct=score_dict, team=team_df["Opponent"][i], opponent=team,
                                                        team_score=int(team_df["Opp"][i]), opponent_score=int(team_df["Tm"][i]))
        
                else:
        
                    if team_df["Outcome"][i] == "W":
        
                        # Current team is home team
                        # score_dict["Home"].append(team)
                        # score_dict["Away"].append(team_df["Opponent"][i])
                        # score_dict["Home_Score"].append(int(team_df["Tm"][i]))
                        # score_dict["Away_Score"].append(int(team_df["Opp"][i]))
                        score_dict = set_massey_score_entry(dct=score_dict, team=team, opponent=team_df["Opponent"][i],
                                                            team_score=int(team_df["Tm"][i]), opponent_score=int(team_df["Opp"][i]))
        
                    else:
        
                        # Opponent team is away team
                        # score_dict["Home"].append(team_df["Opponent"][i])
                        # score_dict["Away"].append(team)
                        # score_dict["Home_Score"].append(int(team_df["Opp"][i]))
                        # score_dict["Away_Score"].append(int(team_df["Tm"][i]))
                        score_dict = set_massey_score_entry(dct=score_dict, team=team_df["Opponent"][i], opponent=team,
                                                            team_score=int(team_df["Opp"][i]), opponent_score=int(team_df["Tm"][i]))

    # Convert to data frame
    score_df = pd.DataFrame(score_dict)

    return score_df

In [3]:
score_df = set_massey_rating_data_frame(filename=FILENAME)

In [4]:
# Get unique teams and index them
teams = list(set(score_df["Home"]).union(set(score_df["Away"])))
team_index = {team: i for i, team in enumerate(teams)}
N = len(teams)

# Initialize Massey matrix and score vector
M = np.zeros((N, N))
b = np.zeros(N)

# Fill the matrix and score vector
for _, row in score_df.iterrows():
    h, a = team_index[row["Home"]], team_index[row["Away"]]
    home_margin = row["Home_Score"] - row["Away_Score"]

    M[h, h] += 1
    M[a, a] += 1
    M[h, a] -= 1
    M[a, h] -= 1

    b[h] += home_margin
    b[a] -= home_margin

# Replace last row to enforce sum constraint (makes matrix invertible)
M[-1, :] = 1
b[-1] = 0

In [5]:
ratings = np.linalg.solve(M, b)

# Convert ratings to a dictionary
massey_ratings = {team: rating for team, rating in zip(teams, ratings)}

# Sort and display rankings
massey_rankings = sorted(massey_ratings.items(), key=lambda x: x[1], reverse=True)

for rank, (team, rating) in enumerate(massey_rankings, 1):
    print(f"{rank}. {team}: {rating:.2f}")

1. Houston: 46.91
2. Arizona: 46.03
3. Purdue: 45.44
4. Connecticut: 45.40
5. Auburn: 43.98
6. Iowa State: 43.56
7. Tennessee: 42.44
8. Alabama: 41.80
9. North Carolina: 41.64
10. Brigham Young: 40.94
11. Duke: 40.88
12. Baylor: 40.74
13. Creighton: 40.71
14. Illinois: 40.50
15. Marquette: 40.25
16. Michigan State: 38.52
17. Kentucky: 38.27
18. Gonzaga: 37.98
19. Kansas: 37.97
20. Saint Mary's (CA): 37.74
21. Wisconsin: 37.44
22. St. John's (NY): 37.01
23. Colorado: 36.67
24. New Mexico: 36.67
25. Texas: 36.63
26. Texas Tech: 36.54
27. TCU: 36.26
28. Cincinnati: 36.13
29. Florida: 36.12
30. Utah: 35.93
31. San Diego State: 35.81
32. Oklahoma: 35.74
33. Nebraska: 35.72
34. Mississippi State: 35.59
35. Clemson: 35.58
36. Wake Forest: 35.54
37. Pittsburgh: 35.43
38. Villanova: 35.05
39. Washington State: 34.94
40. Boise State: 34.57
41. Ohio State: 34.54
42. Florida Atlantic: 34.48
43. Colorado State: 34.25
44. Dayton: 34.08
45. Nevada: 34.05
46. Indiana State: 33.96
47. Iowa: 33.89
48. T